In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
warnings.filterwarnings("ignore", category =UserWarning,)

import gc
import pickle
from collections import OrderedDict
from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import time

from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)

yrs_range= '2002_2020'
em = 'bc006'

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Read in a sample cube for finding the location of gauge in grid
sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr200601.nc')[0][1,:,:]

for yr in range(2015,2021):
    print(yr)
    ######################################################
    ### Get all the data for one year, into one cube
    # (if it already exists in a pickle file, then load it from there)
    ######################################################
    general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{yr}*'
    pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/UKCP18_30mins_{em}/WholeYear/cube_{yr}.pkl"

    if os.path.exists(pickle_file_filepath):
        print("Pickle file exists, so loading that")
        full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
    else:
        print("Pickle file doesnt exist, so creating and then saving that")

        ### Get the data filepaths
        print(f"Loading data for year {yr}")

        # Create cube list
        cubes = load_files_to_cubelist(yr, general_filename)

        # Join them into one (with error handling to deal with times which are wrong)
        try:
            full_year_cube = cubes.concatenate_cube()
            print("Concatenation successful!")
        except Exception as e:
            print(f"Initial concatenation failed: {str(e)}")

            # If initial concatenation fails, remove problematic cubes and try again
            try:
                full_year_cube = remove_problematic_cubes(cubes)
                print("Concatenation successful after removing problematic cubes!")
            except RuntimeError as e:
                print(f"Concatenation failed after removing problematic cubes: {str(e)}")               
        save_cube_as_pickle_file(full_year_cube, pickle_file_filepath)
        
        
    gauge_nums = range(0,1294)
    # Function to process each gauge
    for gauge_num in gauge_nums:
        if not gauge_num in [423, 444, 827, 888]:
                print(f"year is {yr}, gauge num is {gauge_num}")             

                base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/WholeYear"
                # Create the directory if it doesnt exist
                if not os.path.isdir(base_dir):
                    os.makedirs(base_dir)

                ######################################################
                ## Check if any files are missing, across the 3 filtering options
                # If there are: code will continue to run
                # If not: code will move to next gauge
                ######################################################                
                # Create a flag to record whether we are missing any of the files we need
                missing_files = False

                # Check if we are missing any of the files, and if so, change the flag to True
                if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                    missing_files = True

                # If we are missing some files then get the data for the grid cell, 
                if missing_files:
                    # Find location
                    Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

                    # Extract data for the specified indices
                    start= time.time()
                    one_location_cube = full_year_cube[:, idx_2d[0], idx_2d[1]]
                    data = one_location_cube.data
                    end=time.time()
                    print(f"Time to load data is {round(end-start,2)} seconds")

                    ##### Filter cube according to different options
                    # Convert to dataframe
                    df = pd.DataFrame(data, columns=['precipitation (mm/hr)'])
                    df['times'] = one_location_cube.coord('time').units.num2date(one_location_cube.coord('time').points)
                    df['precipitation (mm)'] = df['precipitation (mm/hr)'] / 2   

                    # Search dataframe for events corresponding to durations
                    for duration in [0.5, 1, 2, 3, 6, 12, 24]:

                        filename =  f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv"
                        if not os.path.exists(filename):
                            print(f"Finding the AMAX for {duration}hr events for gauge {gauge_num} in year {yr}")
                            # Find events
                            events_v2 = search_for_valid_events(df, duration=duration, Tb0=Tb0)

                            # Save events to CSV
                            for num, event in enumerate(events_v2):
                                if len(event) > 1:
                                        event.to_csv(f"{base_dir}/{duration}hrs_{yr}_v2_part{num}.csv")
                                        if event['precipitation (mm/hr)'].isna().any():
                                            print("NANs in this event")
                        else:
                            # print(f"already exists{filename}")
                            pass   
                else:
                    pass
                    # print("already exist")



2015
Pickle file exists, so loading that
year is 2015, gauge num is 0
year is 2015, gauge num is 1
year is 2015, gauge num is 2
year is 2015, gauge num is 3
year is 2015, gauge num is 4
year is 2015, gauge num is 5
year is 2015, gauge num is 6
year is 2015, gauge num is 7
year is 2015, gauge num is 8
year is 2015, gauge num is 9
year is 2015, gauge num is 10
year is 2015, gauge num is 11
year is 2015, gauge num is 12
year is 2015, gauge num is 13
year is 2015, gauge num is 14
year is 2015, gauge num is 15
year is 2015, gauge num is 16
year is 2015, gauge num is 17
year is 2015, gauge num is 18
year is 2015, gauge num is 19
year is 2015, gauge num is 20
year is 2015, gauge num is 21
year is 2015, gauge num is 22
year is 2015, gauge num is 23
year is 2015, gauge num is 24
year is 2015, gauge num is 25
year is 2015, gauge num is 26
year is 2015, gauge num is 27
year is 2015, gauge num is 28
year is 2015, gauge num is 29
year is 2015, gauge num is 30
year is 2015, gauge num is 31
year is 2

year is 2015, gauge num is 279
year is 2015, gauge num is 280
year is 2015, gauge num is 281
year is 2015, gauge num is 282
year is 2015, gauge num is 283
year is 2015, gauge num is 284
year is 2015, gauge num is 285
year is 2015, gauge num is 286
year is 2015, gauge num is 287
year is 2015, gauge num is 288
year is 2015, gauge num is 289
year is 2015, gauge num is 290
year is 2015, gauge num is 291
year is 2015, gauge num is 292
year is 2015, gauge num is 293
year is 2015, gauge num is 294
year is 2015, gauge num is 295
year is 2015, gauge num is 296
year is 2015, gauge num is 297
year is 2015, gauge num is 298
year is 2015, gauge num is 299
year is 2015, gauge num is 300
year is 2015, gauge num is 301
year is 2015, gauge num is 302
year is 2015, gauge num is 303
year is 2015, gauge num is 304
year is 2015, gauge num is 305
year is 2015, gauge num is 306
year is 2015, gauge num is 307
year is 2015, gauge num is 308
year is 2015, gauge num is 309
year is 2015, gauge num is 310
year is 

(163, 305)
Time to load data is 42.53 seconds
Finding the AMAX for 0.5hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 13.617813110351562
Finding the AMAX for 1hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 13.617813110351562
Finding the AMAX for 2hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 42.83283233642578
Finding the AMAX for 3hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 42.83283233642578
Finding the AMAX for 6hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 39.585411071777344
Finding the AMAX for 12hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 39.585411071777344
Finding the AMAX for 24hr events for gauge 414 in year 2015
Event doesnt contain NAN, total event precip is 72.04560089111328
year is 2015, gauge num is 415
(51, 317)
Time to load data is 16.76 sec

(38, 317)
Time to load data is 11.31 seconds
Finding the AMAX for 0.5hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 28.323057174682617
Finding the AMAX for 1hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 28.323057174682617
Finding the AMAX for 2hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 28.323057174682617
Finding the AMAX for 3hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 28.493804931640625
Finding the AMAX for 6hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 28.494504928588867
Finding the AMAX for 12hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 47.62392807006836
Finding the AMAX for 24hr events for gauge 424 in year 2015
Event doesnt contain NAN, total event precip is 49.67332077026367
year is 2015, gauge num is 425
(38, 321)
Time to load data is 13.17 sec

(168, 314)
Time to load data is 11.49 seconds
Finding the AMAX for 0.5hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 12.727354049682617
Finding the AMAX for 1hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 19.642189025878906
Finding the AMAX for 2hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 19.642189025878906
Finding the AMAX for 3hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 19.790424346923828
Finding the AMAX for 6hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 47.95597839355469
Finding the AMAX for 12hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 47.95597839355469
Finding the AMAX for 24hr events for gauge 433 in year 2015
Event doesnt contain NAN, total event precip is 47.95597839355469
year is 2015, gauge num is 434
(57, 289)
Time to load data is 14.59 sec

...

Finding the AMAX for 24hr events for gauge 12 in year 2016
2 events
Event doesnt contain NAN, total event precip is 7.349653720855713
year is 2016, gauge num is 13
(255, 302)
Time to load data is 128.54 seconds
Finding the AMAX for 0.5hr events for gauge 13 in year 2016
Event doesnt contain NAN, total event precip is 28.68621063232422
Finding the AMAX for 1hr events for gauge 13 in year 2016
Event doesnt contain NAN, total event precip is 42.23850631713867
Finding the AMAX for 2hr events for gauge 13 in year 2016
Event doesnt contain NAN, total event precip is 42.23850631713867
Finding the AMAX for 3hr events for gauge 13 in year 2016
Event doesnt contain NAN, total event precip is 42.23850631713867
Finding the AMAX for 6hr events for gauge 13 in year 2016
Event doesnt contain NAN, total event precip is 42.23850631713867
Finding the AMAX for 12hr events for gauge 13 in year 2016
Event doesnt contain NAN, total event precip is 42.271461486816406
Finding the AMAX for 24hr events for gaug

2 events
Event doesnt contain NAN, total event precip is 76.8139419555664
year is 2016, gauge num is 22
(247, 287)
Time to load data is 11.96 seconds
Finding the AMAX for 0.5hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event precip is 16.761144638061523
Finding the AMAX for 1hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event precip is 17.933578491210938
Finding the AMAX for 2hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event precip is 18.487531661987305
Finding the AMAX for 3hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event precip is 33.50556564331055
Finding the AMAX for 6hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event precip is 40.534908294677734
Finding the AMAX for 12hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event precip is 40.632015228271484
Finding the AMAX for 24hr events for gauge 22 in year 2016
Event doesnt contain NAN, total event p

(241, 295)
Time to load data is 11.35 seconds
Finding the AMAX for 0.5hr events for gauge 31 in year 2016
Event doesnt contain NAN, total event precip is 59.8515510559082
Finding the AMAX for 1hr events for gauge 31 in year 2016
Event doesnt contain NAN, total event precip is 59.8515510559082
Finding the AMAX for 2hr events for gauge 31 in year 2016
Event doesnt contain NAN, total event precip is 59.8515510559082
Finding the AMAX for 3hr events for gauge 31 in year 2016
Event doesnt contain NAN, total event precip is 59.8515510559082
Finding the AMAX for 6hr events for gauge 31 in year 2016
Event doesnt contain NAN, total event precip is 59.8515510559082
Finding the AMAX for 12hr events for gauge 31 in year 2016
Event doesnt contain NAN, total event precip is 60.100059509277344
Finding the AMAX for 24hr events for gauge 31 in year 2016
2 events
Event doesnt contain NAN, total event precip is 60.32464599609375
year is 2016, gauge num is 32
(241, 304)
Time to load data is 11.73 seconds
F

year is 2016, gauge num is 40
(231, 302)
Time to load data is 37.17 seconds
Finding the AMAX for 0.5hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 13.100041389465332
Finding the AMAX for 1hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 25.901330947875977
Finding the AMAX for 2hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 26.249656677246094
Finding the AMAX for 3hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 26.249656677246094
Finding the AMAX for 6hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 45.32917785644531
Finding the AMAX for 12hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 45.32918167114258
Finding the AMAX for 24hr events for gauge 40 in year 2016
Event doesnt contain NAN, total event precip is 85.2919692993164
year is 2016, gauge num is 41
(228, 297)
Time to 

Event doesnt contain NAN, total event precip is 43.04154586791992
year is 2016, gauge num is 49
(221, 338)
Time to load data is 40.96 seconds
Finding the AMAX for 0.5hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is 14.406855583190918
Finding the AMAX for 1hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is 15.165094375610352
Finding the AMAX for 2hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is 29.100688934326172
Finding the AMAX for 3hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is 29.100688934326172
Finding the AMAX for 6hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is 29.54872703552246
Finding the AMAX for 12hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is 30.065250396728516
Finding the AMAX for 24hr events for gauge 49 in year 2016
Event doesnt contain NAN, total event precip is